In [ ]:
import pandas as pd
import numpy as np
from tqdm import  tqdm

In [ ]:
company=pd.read_excel('companyInfo.xlsx')
RPT1=pd.read_excel('RPT_Operation.xlsx',sheet_name=1)
RPT2=pd.read_excel('RPT_Operation1.xlsx',sheet_name=1)

In [ ]:
listed_RPT_info=pd.DataFrame(columns=['cp1','cp2','relation','tradeYear'])
k=0
unlisted=0
for i in tqdm(RPT1.index):
    cp1=RPT1.loc[i,'Stkcd']
    cp2_name=RPT1.loc[i,'Repart']
    cp2_name= cp2_name.replace('（','(')
    cp2_name= cp2_name.replace('）',')')
    cp2=company.loc[[str(j) in cp2_name or cp2_name in str(j) for j in company['FullName'].values]]
    if len(cp2)==0:
        unlisted+=1
        continue
    cp2=cp2.iloc[0,2]
    listed_RPT_info.loc[k,:]=[cp1,cp2,RPT1.loc[i,'Relation'],RPT1.loc[i,'Trddt'][0:4]]
    k+=1

In [ ]:
for i in tqdm(RPT2.index):
    cp1=RPT2.loc[i,'Stkcd']
    cp2_name=RPT2.loc[i,'Repart']
    cp2_name= cp2_name.replace('（','(')
    cp2_name= cp2_name.replace('）',')')
    cp2=company.loc[[str(j) in cp2_name or cp2_name in str(j) for j in company['FullName'].values]]
    if len(cp2)==0:
        unlisted+=1
        continue
    cp2=cp2.iloc[0,2]
    listed_RPT_info.loc[k,:]=[cp1,cp2,RPT2.loc[i,'Relation'],RPT2.loc[i,'Trddt'][0:4]]
    k+=1

In [ ]:
listed_RPT_info.to_csv('./listed_RPT.csv',index=False)

In [ ]:
listed_RPT_info=pd.read_csv('./listed_RPT.csv')
lisetd_comp=set(listed_RPT_info['cp1'])|set(listed_RPT_info['cp2'])

In [ ]:
def f(df):
    return df.iloc[0,:]
def preprocess(df:pd.DataFrame,lisetd_comp,nan_threhold=0.15,have_type=True):
    df['year']=df['Accper'].apply(lambda x:x[0:4])
    df=df.loc[df['year']>='2000']
    total_num=len(df)
    na_col=df.isna().sum().apply(lambda x:x/total_num)
    df=df[na_col.index[na_col<nan_threhold]]
    df=df.loc[[i in lisetd_comp for i in df['Stkcd']]]
    df=df.loc[[i[5:7]=='12' for i in df['Accper']]]
    if have_type:
        #df=df.loc[df['Typrep']=='A']
        df=df.groupby(['Stkcd','year']).apply(f)
        df.reset_index(drop=True,inplace=True)
    return df

In [ ]:
debt_ability=preprocess(debt_ability,lisetd_comp)
develop_ability=preprocess(develop_ability,lisetd_comp)
risk_ratio=preprocess(risk_ratio,lisetd_comp)

In [ ]:
debt_ability=pd.read_excel('./偿债能力185738156/FI_T1.xlsx')
debt_ability=preprocess(debt_ability,lisetd_comp)

In [ ]:
develop_ability=pd.read_excel('./发展能力190527993/FI_T8.xlsx')
develop_ability=preprocess(develop_ability,lisetd_comp)

In [ ]:
risk_ratio=pd.read_excel('./风险水平190314961/FI_T8.xlsx')
risk_ratio=preprocess(risk_ratio,lisetd_comp)

In [ ]:
X=debt_ability.merge(develop_ability,how='outer',on=['Stkcd','year']).merge(risk_ratio,how='outer',on=['Stkcd','year'])

In [ ]:
manage_ability=pd.read_excel('./经营能力185956829/FI_T4.xlsx')
manage_ability=preprocess(manage_ability,lisetd_comp)


In [ ]:
X=X.merge(manage_ability,how='outer',on=['Stkcd','year'])

In [ ]:
stock_ratio=pd.read_excel('./每股指标190641029/FI_T9.xlsx')
stock_ratio=preprocess(stock_ratio,lisetd_comp)
#X=X.merge(stock_ratio,how='outer',on=['Stkcd','year'])

In [ ]:
col=X.columns
col=list(col[['Accper' not in i and 'Typrep' not in i and 'Indcd' not in i and 'Indnme' not in i  for i in col]])

In [ ]:
special_ratio=pd.read_excel('./上市公司基本信息特色指标表185734588/AF_CFEATUREPROFILE.xlsx')
special_ratio=preprocess(special_ratio,lisetd_comp)
# X=X.merge(special_ratio,how='outer',on=['Stkcd','year'])
# X=X.loc[X['year']>='2000']

In [ ]:
cash_ratio=pd.read_excel('./现金流分析190426810/FI_T6.xlsx')
cash_ratio=preprocess(cash_ratio,lisetd_comp)
#X=X.merge(cash_ratio,how='outer',on=['Stkcd','year'])

In [ ]:
related_value=pd.read_excel('./相对价值指标190851871/FI_T10.xlsx')
related_value=preprocess(related_value,lisetd_comp,have_type=False)
#X=X.merge(related_value,on=['Stkcd','year'],how='outer')

In [ ]:
#profit_ability=pd.read_excel('./盈利能力190417749/FI_T5.xlsx')
profit_ability=preprocess(profit_ability,lisetd_comp)
#X=X.merge(profit_ability,on=['Stkcd','year'],how='outer')

In [ ]:
X=debt_ability.merge(develop_ability,how='outer',on=['Stkcd','year']).merge(risk_ratio,how='outer',on=['Stkcd','year'])
X=X.merge(manage_ability,how='outer',on=['Stkcd','year'])
X=X.merge(stock_ratio,how='outer',on=['Stkcd','year'])
X=X.merge(special_ratio,how='outer',on=['Stkcd','year'])
X=X.loc[X['year']>='2000']
X=X.merge(cash_ratio,how='outer',on=['Stkcd','year'])
X=X.merge(related_value,on=['Stkcd','year'],how='outer')
X=X.merge(profit_ability,on=['Stkcd','year'],how='outer')

In [ ]:
year_hs=pd.read_excel('./个股换手率表（年）192917112/LIQ_TOVER_Y.xlsx')
X['year']=X['year'].apply(lambda x: int(x))
X=X.merge(year_hs,left_on=['Stkcd','year'],right_on=['Stkcd','Trdynt'],how='left')

In [ ]:
month_hs=pd.read_excel('./个股换手率表（月）193041231/LIQ_TOVER_M.xlsx')
month_hs['year']=month_hs['Trdmnt'].apply(lambda x:x[0:4])

In [ ]:
mpnth_hs_var=month_hs.groupby(['Stkcd','year']).apply(np.var)

In [ ]:
mpnth_hs_var['year']=[int(i[1]) for i in mpnth_hs_var.index.values]
mpnth_hs_var['Stkcd']=[i[0] for i in mpnth_hs_var.index.values]
mpnth_hs_var.reset_index(drop=True,inplace=True)

In [ ]:
X=X.merge(mpnth_hs_var,on=['year','Stkcd'],how='left')

In [ ]:
year_hs=pd.read_excel('./年个股回报率文件192623976/TRD_Year.xlsx')
X['year']=X['year'].apply(lambda x: int(x))
X=X.merge(year_hs,left_on=['Stkcd','year'],right_on=['Stkcd','Trdynt'],how='left')

In [ ]:
month_pr=pd.read_excel('./月个股回报率文件192820634/TRD_Mnth.xlsx')
month_pr['year']=month_pr['Trdmnt'].apply(lambda x:x[0:4])
month_pr_var=month_pr.groupby(['Stkcd','year']).apply(np.var)
month_pr_var['year']=[int(i[1]) for i in month_pr_var.index.values]
month_pr_var['Stkcd']=[i[0] for i in month_pr_var.index.values]
month_pr_var.reset_index(drop=True,inplace=True)
X=X.merge(mpnth_hs_var,on=['year','Stkcd'],how='left')


In [ ]:
col=X.columns
col=list(col[['Accper' not in i and 'Typrep' not in i and 'Indcd' not in i and 'Indnme' not in i  for i in col]])
X=X[col]
X.to_csv('fiancial_features.csv',index=False)

In [ ]:
listed_RPT_info=pd.read_csv('listed_RPT.csv')

In [ ]:
import networkx as nx
def graph_cal(df):
    edges=df[['cp1','cp2']].values
    G=nx.Graph()
    G.add_edges_from(edges)
    connectedArea=[]
    size=[]
    for g in nx.connected_components(G):
        connectedArea.append(g)
        size.append(len(g))
    d={"cc_num":len(connectedArea),'ave_cc_size':np.mean(size),'total_size':len(df)}
    return pd.DataFrame(d,index=[0])

In [ ]:
delete_index=[]
for i in listed_RPT_info.index:
    if listed_RPT_info.loc[i,'cp1']==listed_RPT_info.loc[i,'cp2']:
        delete_index.append(i)
listed_RPT_info.drop(delete_index,inplace=True)

In [ ]:
info=listed_RPT_info.groupby('tradeYear').apply(graph_cal)
info.to_csv('rpt_statistics.csv')

In [35]:
features=X

In [34]:
def preprocess(df:pd.DataFrame,lisetd_comp,nan_threhold=0.15,have_type=True):
    df['year']=df['Enddate'].apply(lambda x:int(x[0:4]))
    df=df.loc[df['year']>=2000]
    total_num=len(df)
    na_col=df.isna().sum().apply(lambda x:x/total_num)
    df=df[na_col.index[na_col<nan_threhold]]
    df=df.loc[[i in lisetd_comp for i in df['Stkcd']]]
    #df=df.loc[[i[5:7]=='12' for i in df['Accper']]]
    if have_type:
        df=df.loc[df['Typrep']=='A']
    
    return df
manager_features=pd.read_excel('./管理层治理能力191704076/BDT_ManaGovAbil.xlsx')
manager_features=preprocess(manager_features,have_type=False,lisetd_comp=lisetd_comp)

In [36]:
X=features.merge(manager_features,how='left',on=['Stkcd','year'])

In [ ]:
X.to_csv('./financial_features.csv',index=False)

In [38]:
def preprocess(df:pd.DataFrame,lisetd_comp,nan_threhold=0.15,have_type=True):
    df['year']=df['EndDate'].apply(lambda x:int(x[0:4]))
    df=df.loc[df['year']>=2000]
    total_num=len(df)
    na_col=df.isna().sum().apply(lambda x:x/total_num)
    df=df[na_col.index[na_col<nan_threhold]]
    df=df.loc[[i in lisetd_comp for i in df['Stkcd']]]
    #df=df.loc[[i[5:7]=='12' for i in df['Accper']]]
    if have_type:
        df=df.loc[df['Typrep']=='A']
    
    return df
equity_property=pd.read_excel('./中国上市公司股权性质文件181729700/EN_EquityNatureAll.xlsx')
equity_property=preprocess(equity_property,lisetd_comp,have_type=False)
X=X.merge(equity_property,how='left',on=['Stkcd','year'])
X.to_csv('./financial_features.csv',index=False)

In [39]:
X.to_csv('./financial_features.csv',index=False)

In [ ]:
board=pd.read_excel('./董监高个人特征文件220031349/TMT_FIGUREINFO.xlsx')

In [ ]:
import networkx as nx
board['year']=board['Reptdt'].apply(lambda x:int(x[0:4]))
board['PersonID']=board['PersonID'].apply(lambda x: 'P'+str(x))
def graph_cal(df):
    edges=df[['Stkcd','PersonID']].values
    G=nx.Graph()
    G.add_edges_from(edges)
    connectedArea=[]
    size=[]
    comp_size=[]
    for g in nx.connected_components(G):
        comp_set=set()
        connectedArea.append(g)
        for c in g:
            if isinstance(c,int):
                comp_set.add(c)
        size.append(len(g))
        comp_size.append(len(comp_set))
    d={"cc_num":len(connectedArea),'ave_cc_size':np.mean(size),'total_size':len(df),'comp_size':np.mean(comp_size)}
    return pd.DataFrame(d,index=[0])
borad_statistics=board.groupby('year').apply(graph_cal)

In [ ]:
borad_statistics.to_csv('board_statictics.csv')

In [ ]:
board.to_csv('./borad.csv',index=False)